In [1]:
import pandas as pd

In [2]:
cust_dict = {
    'customer_name' : ['Alice', 'Tom', 'James', 'Yerin', 'Min', 'Katherine', 'John', 'Park', 'Maria', 'Derik', 'Jin'],
    'cust_country' : ['US', 'GB', 'US', 'KOR', 'KOR', 'GB', 'US', 'KOR', 'US', 'GB', 'KOR'],
    'gender' : ['F', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'M', 'F'],
    'grade' : [3, 1, 2, 5, 4, 3, 5, 7, 2, 4, 5],
    'age' : [25, 34, 26, 33, 67, 29, 54, 21, 77, 29, 16]
}

customer = pd.DataFrame(cust_dict)

customer.head(11)

,customer_name,cust_country,gender,grade,age
0,Alice,US,F,3,25
1,Tom,GB,M,1,34
2,James,US,M,2,26
3,Yerin,KOR,F,5,33
4,Min,KOR,M,4,67
5,Katherine,GB,F,3,29
6,John,US,M,5,54
7,Park,KOR,M,7,21
8,Maria,US,F,2,77
9,Derik,GB,M,4,29


In [3]:
cust_group = customer.groupby('cust_country')
type(cust_group)

pandas.core.groupby.generic.DataFrameGroupBy

In [4]:
cust_group.head()

,customer_name,cust_country,gender,grade,age
0,Alice,US,F,3,25
1,Tom,GB,M,1,34
2,James,US,M,2,26
3,Yerin,KOR,F,5,33
4,Min,KOR,M,4,67
5,Katherine,GB,F,3,29
6,John,US,M,5,54
7,Park,KOR,M,7,21
8,Maria,US,F,2,77
9,Derik,GB,M,4,29


#### 개별 aggregation 컬럼 및 aggregation 연산별로 API 호출

In [5]:
cust_agg = pd.DataFrame()
cust_agg['sum_grade'] = cust_group['grade'].sum()
cust_agg['max_grade'] = cust_group['grade'].max()
cust_agg.head(11)

,sum_grade,max_grade
cust_country,,
GB,8,4
KOR,21,7
US,12,5


In [7]:
cust_agg['avg_age'] = cust_group['age'].mean()
cust_agg['max_age'] = cust_group['age'].max()
cust_agg.head()

,sum_grade,max_grade,avg_age,max_age
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


In [8]:
# groupby 키 컬럼은 DataFrame의 인덱스로 변환
print(cust_agg.index.name)
cust_agg = cust_agg.reset_index()
cust_agg.head()

cust_country


,cust_country,sum_grade,max_grade,avg_age,max_age
0,GB,8,4,30.666667,34
1,KOR,21,7,34.250000,67
2,US,12,5,45.500000,77


#### agg()함수를 활용. agg()인자로 aggregation 함수 리스트를 입력. 여전히 aggregation 컬럼 별로 API 적용

In [9]:
cust_agg1 = cust_group['grade'].agg(['sum', 'max'])
cust_agg2 = cust_group['age'].agg(['mean', 'max'])
print(cust_agg1.head())
print(cust_agg2.head())

              sum  max
cust_country          
GB              8    4
KOR            21    7
US             12    5
                   mean  max
cust_country                
GB            30.666667   34
KOR           34.250000   67
US            45.500000   77


In [10]:
cust_agg = cust_agg1.merge(cust_agg2, on='cust_country', how='left')
cust_agg.head()

,sum,max_x,mean,max_y
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


In [14]:
cust_agg = pd.DataFrame()
cust_agg[['sum_grade', 'max_grade']] = cust_group['grade'].agg(['sum', 'max'])
cust_agg[['avg_grade', 'max_grade']] = cust_group['age'].agg(['mean', 'max'])
cust_agg.head()

,sum_grade,max_grade,avg_grade
cust_country,,,
GB,8,34,30.666667
KOR,21,67,34.250000
US,12,77,45.500000


#### agg()에 인자로 Aggregation 컬럼값과 Aggregation함수 리스트를 기재한 dictionary 값 입력

In [11]:
agg_dict = {
    'grade' : ['sum', 'max'],
    'age' : ['mean', 'max']
}
cust_agg = cust_group.agg(agg_dict)
cust_agg.head()

grade            age    
               sum max       mean max
cust_country                         
GB               8   4  30.666667  34
KOR             21   7  34.250000  67
US              12   5  45.500000  77

#### agg()에 인자로 Aggrgation 컬럼값과 Aggrgation 함수 리스트를 기재한 dictionary 값 입력


In [20]:
agg_dict = {
    'grade' : ['sum', 'max'],
    'age' : ['mean', 'max']
}
cust_agg = cust_group.agg(agg_dict)
cust_agg.head()

grade            age    
               sum max       mean max
cust_country                         
GB               8   4  30.666667  34
KOR             21   7  34.250000  67
US              12   5  45.500000  77

#### groupby 수행 결과 DataFrame의 컬럼명 변경

In [21]:
cust_agg.columns = [('_').join(column) for column in cust_agg.columns]

In [22]:
cust_agg.head()

,grade_sum,grade_max,age_mean,age_max
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


#### case when 조건으로 각각 filtering 하여 별도의 DataFrame생성

In [52]:
cond_male = customer['gender'] == 'M'
cond_female = customer['gender'] == 'F'
cust_male = customer[cond_male]
cust_female = customer[cond_female]

In [53]:
cust_male

,customer_name,cust_country,gender,grade,age
1,Tom,GB,M,1,34
2,James,US,M,2,26
4,Min,KOR,M,4,67
6,John,US,M,5,54
7,Park,KOR,M,7,21
9,Derik,GB,M,4,29


#### 원본 데이터에 cust_country로 groupby수행

In [54]:
cust_agg = customer.groupby('cust_country')['grade'].sum()
cust_agg = cust_agg.reset_index()

In [55]:
cust_agg.head()

,cust_country,grade
0,GB,8
1,KOR,21
2,US,12


#### filtering된 데이터에 cust_country로 groupby수행

In [56]:
cust_male_agg = cust_male.groupby('cust_country')['grade'].sum()
cust_female_agg = cust_female.groupby('cust_country')['grade'].sum()
cust_male_agg = cust_male_agg.reset_index()
cust_female_agg = cust_female_agg.reset_index()

In [57]:
cust_male_agg

,cust_country,grade
0,GB,5
1,KOR,11
2,US,7


#### 원본 데이터에 groupby된 데이터 세트와 filtering된 데이터에 groupby된 데이터 세트를 조인

In [58]:
cust_agg = cust_agg.merge(cust_male_agg, on='cust_country', how='left')
cust_agg = cust_agg.merge(cust_female_agg, on='cust_country', how='left')
cust_agg.head()

,cust_country,grade_x,grade_y,grade
0,GB,8,5,3
1,KOR,21,11,10
2,US,12,7,5


In [59]:
cust_agg.colunms = ['cust_country', 'sum_grade', 'sum_male_grade', 'sum_female_grade']


In [60]:
cust_agg.head()

,cust_country,grade_x,grade_y,grade
0,GB,8,5,3
1,KOR,21,11,10
2,US,12,7,5
